In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Data pre-process

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test  = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
submission = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
train['processed_text'] = train['excerpt']
test['processed_text'] =  test['excerpt']

In [ ]:
# Apostrophe Dictionary(Từ điển viết tắt)
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

# Function replace acronyms (Hàm xử lý từ viết tắt -> từ đầy đủ)
def lookup_dict(text, dictionary):
    for word in text.split():
        if word.lower() in dictionary:
            if word.lower() in text.split():
                text = text.replace(word, dictionary[word.lower()])
    return text

In [ ]:
# Using lambda function to handles predefined dictionary abbreviations and assign value to new column
# Sử dụng lambda function để tiền xử lý dữ liệu thay thế các từ viết tắt bằng các từ đầy đủ của chúng và gán giá trị đã xử lý vào cột mới
train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x,apostrophe_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x,apostrophe_dict))

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords') 
ps = PorterStemmer()
words = stopwords.words("english")

train['processed_text'] = train['excerpt'].apply(lambda x: " ".join([ps.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
test['processed_text'] = test['excerpt'].apply(lambda x: " ".join([ps.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [ ]:
import string

# Make all words lower case
train['processed_text'] = train['processed_text'].str.lower()
test['processed_text'] = test['processed_text'].str.lower()

# Remove punctuation
table = str.maketrans('', '', string.punctuation)
train['processed_text'] = [train['processed_text'][row].translate(table) for row in range(len(train['processed_text']))]
test['processed_text'] = [test['processed_text'][row].translate(table) for row in range(len(test['processed_text']))]

# Remove hash tags
train['processed_text'] = train['processed_text'].str.replace("#", " ")
test['processed_text'] = test['processed_text'].str.replace("#", " ")

# Remove words less than 1 character
train['processed_text'] = train['processed_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
test['processed_text'] = test['processed_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))

In [ ]:
# Remove rare words (Xóa những từ hiếm gặp)
from collections import Counter
from itertools import chain

# split words into lists
v = train['processed_text'].str.split().tolist() 
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
train['processed_text'] = [' '.join([j for j in i if c[j] > 1]) for i in v]

# split words into lists
v = test['processed_text'].str.split().tolist() 
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
test['processed_text'] = [' '.join([j for j in i if c[j] > 1]) for i in v]

In [ ]:
y=train['target']
X=train['processed_text']
X_test=test['processed_text']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Feature extraction
# Using stop_word into TfidfVectorizer (Term-Frequency Inverse Document-Frequency)
vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
train_tfIdf = vectorizer_tfidf.fit_transform(X_train.values.astype('U'))
val_tfIdf = vectorizer_tfidf.transform(X_val.values.astype('U'))
X_test_tfIdf = vectorizer_tfidf.transform(X_test.values.astype('U'))
print(vectorizer_tfidf.get_feature_names()[:5])

### Define model

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor as GP

# Build model using GaussianProcessRegressor 
# Using GaussianProcessRegressor because GP regression works well on small datasets by assuming the data points lie on a multi-dimensional distribution (Multivariate Normal))
model = GP(random_state=1).fit(train_tfIdf.todense(), y_train)
print(model.score(train_tfIdf.todense(), y_train))
y_pred = model.predict(val_tfIdf.todense())
model.score(val_tfIdf.todense(), y_val)

In [ ]:
predictions = model.predict(X_test_tfIdf.todense())
submission.target = predictions
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission